In [57]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np
import sklearn
from sklearn import preprocessing
from tqdm import tqdm
import scipy
import itertools
import collections
import time

In [25]:
df = pd.read_pickle('/content/drive/MyDrive/ADM/Homework4/df.pkl')

In [58]:
def get_customers(df):
  customers = df.drop_duplicates(subset='CustomerID')
  customers = customers.drop(['TransactionID', 'CustomerID'], axis=1)
  return customers

# one hot encoding
def encoder(df):
  # CustomerDOB
  df.CustomerDOB = pd.qcut(df.CustomerDOB.apply(lambda x: x.year), 3, labels = ['age_range_1' , 'age_range_2', 'age_range_3'])
  # CustLocation
  counts = df.CustLocation.value_counts()
  min_frequency = 10
  repl = counts[counts <= min_frequency].index
  df.CustLocation = df.CustLocation.replace(repl, 'others')
  # CustAccountBalance
  df.CustAccountBalance = pd.qcut(df.CustAccountBalance, 3, labels = ['poor' , 'wealthy', 'rich'])
  # TransactionTime
  df.TransactionTime = pd.cut(df.TransactionTime.apply(lambda x: int(x.hour)), bins=[0, 13, 20, 24], labels = ['morning', 'afternoon', 'evening'])
  # TransactionAmount (INR)
  df['TransactionAmount (INR)'] = pd.qcut(df['TransactionAmount (INR)'], 3, labels = ['low', 'medium', 'high'])
  # one hot encoding
  df = pd.get_dummies(df)
  return df

from numpy.core.numeric import outer
def compute_signature(l, hash_functions, D):
  signature = []
  for perm in hash_functions:
    a,b = perm
    min_hash = float('inf')
    for el in l:
      hash = (a*el + b) % D
      if hash < min_hash:
        min_hash = hash
    signature.append(min_hash)
  return signature


def MinHash(df, seed):
  N = 100   #number of hash functions
  D = df.shape[1]   #number of indeces
  minimum = 0    # inclusive
  maximum = D    # exclusive
  signature_matrix = np.zeros([len(df), N], dtype = np.int32)
  p = 2943
  np.random.seed(seed)
  hash_functions = [(np.random.randint(minimum, maximum),np.random.randint(minimum, maximum)) for _ in range(N)]
  df = scipy.sparse.lil_matrix(df)
  for i in tqdm(range(df.shape[0])):
    signature_matrix[i] = compute_signature(df.rows[i], hash_functions, D)
  return signature_matrix

def create_buckets(sig_mat, b, r):
    d, n = sig_mat.shape
    assert(n==b*r)
    buckets = collections.defaultdict(set)
    bands = np.array_split(sig_mat, b, axis=1)
    for i,band in tqdm(enumerate(bands)):
        for j in range(d):
            band_id = tuple(list(band[j,:])+[str(i)])
            buckets[band_id].add(j)
    return buckets

# function that estimate the jaccard similarity
def J_estimate(signature_matrix, user_index1, user_index2):
  return (signature_matrix[user_index1, :] == signature_matrix[user_index2, :]).mean()

# function that find similar customers to a given customer
def find_similars(user_index, buckets):
  similar_users = []
  for value in buckets.values():
    if user_index in value:
      similar_users = similar_users + list(value)
  similar_users = np.unique(np.array(similar_users))
  return similar_users

def compute_threshold(b, r):
  return (1/b)**(1/r)

We choose values of b and r such that the threshold we obtain is very high, specifically around 80%. We see that for b=10 and r=10 the value is about 0.79

In [42]:
compute_threshold(b = 10, r = 10)

0.7943282347242815

In [6]:
query = pd.read_pickle('/content/drive/MyDrive/ADM/Homework4/query_processed.pkl')

In [26]:
# merge the dataset with the query (the queries are the first 50 rows)
query_plus_df = pd.concat([query, df])

In [59]:
# preprocess dataset
customers = get_customers(query_plus_df)
customers = encoder(customers)
# LSH
t0 = time.time()
signature_matrix = MinHash(customers, seed=1234)
# np.save('/content/drive/MyDrive/ADM/Homework4/signature_matrix.npy', signature_matrix)
# signature_matrix = np.load('/content/drive/MyDrive/ADM/Homework4/signature_matrix.npy')
buckets = create_buckets(signature_matrix, b = 10, r=10)

# find similar customers to the query
query_result = collections.defaultdict(list)
for i in range(len(query)):
  similars = find_similars(i, buckets)
  query_result[i] = similars[similars > 49]  #remove queries themselves from results
t1 = time.time()

100%|██████████| 884265/884265 [01:38<00:00, 9022.15it/s]
10it [00:27,  2.79s/it]


To see if our LSH implementation works well we calculate the mean of the scores for each query element and then compute the mean of the results, this should be around 0.79.

In [60]:
np.mean(np.array([np.mean(np.array(list(map(lambda x: J_estimate(signature_matrix, i, x), query_result[i])))) for i in range(50)]))

0.7540447180551679

The running time is:

In [65]:
print((t1-t0))

159.63974261283875
